# Model Development
This notebook documents baseline, tree-based, and deep learning models used for building cooling load prediction.

## Baseline Models
- Mean predictor
- Last value predictor
- Linear regression baseline

## Tree-Based Models
- XGBoost
- LightGBM

## Deep Learning Models
- LSTM network
